In [1]:
from multiprocessing import Process
from kodo.service.node import run_service
import httpx
import importlib

from IPython.display import display, HTML

# REGISTRY1 STARTUP

We startup a `registry1` and a `node1` connects to this `registry1`.

Next we startup another `registry2` and a `node2` connects to this`registry2`.

In [2]:
registry1ip = "http://localhost:3366"
node1ip = "http://localhost:3367"
registry2ip = "http://localhost:3368"
node2ip = "http://localhost:3369"

In [3]:
registry1kw = {
    "url": registry1ip, 
    "organization": "registry1",
    "node": False, 
    "registry": True,
    # initialise the cache, which we will use later
    "cache": "./registry1.json",
    # start from scratch
    "reset": True
}
registry1 = Process(target=run_service, kwargs=registry1kw)
registry1.start()

INFO - 2024-12-26 18:06:12,383 - kodo - node - http://localhost:3366: registry startup
INFO - 2024-12-26 18:06:12,383 - kodo - node - http://localhost:3366: ignore cache ./registry1.json
INFO - 2024-12-26 18:06:12,383 - kodo - node - http://localhost:3366: successfully started registry
INFO - 2024-12-26 18:06:12,383 - uvicorn.error - server - Started server process [76024]
INFO - 2024-12-26 18:06:12,383 - uvicorn.error - on - Waiting for application startup.
INFO - 2024-12-26 18:06:12,385 - uvicorn.error - on - Application startup complete.


In [20]:
httpx.get(f"{registry1ip}/flows").json()

{'total': 0,
 'filtered': 0,
 'p': 0,
 'pp': 10,
 'items': [],
 'by': 'name, node, url',
 'q': None}

In [21]:
display(HTML(httpx.get(f"{registry1ip}/flows?format=html").content.decode("utf-8")))

,registry,node,organization,created,modified,heartbeat,url,name,author,description,tags


### simulate node connection
ignore duplicate flows

In [51]:
data = {
    "url": "http://localhost:3380",
    "flows": [
        {
            "url": "/test1",
            "name": "Test 1"
        },
        {
            "url": "/test2",
            "name": "Test 2"
        },
        {
            "url": "/test2",
            "name": "Test 3"
        },
        {
            "url": "/test4",
            "name": "Test 4"
        },
    ]
}
resp = httpx.post(f"{registry1ip}/register", json=data)

resp.json()

{'url': 'http://localhost:3366',
 'organization': None,
 'connection': {},
 'node': True,
 'registry': True,
 'provider': False,
 'started_at': '2024-12-26T18:22:01.033559',
 'idle': True,
 'now': '2024-12-26T18:22:07.114229',
 'message': ['registered node http://localhost:3380 (first visit)',
  'received 4 flows, registered 3 flows']}

In [52]:
display(HTML(httpx.get(f"{registry1ip}/flows?format=html").content.decode("utf-8")))

,registry,node,organization,created,modified,heartbeat,url,name,author,description,tags
0,http://localhost:3366,http://localhost:3380,None,2024-12-26 18:22:07.113278,2024-12-26 18:22:07.113278,2024-12-26 18:22:07.113278,/test1,Test 1,missing author,missing description,[]
1,http://localhost:3366,http://localhost:3380,None,2024-12-26 18:22:07.113278,2024-12-26 18:22:07.113278,2024-12-26 18:22:07.113278,/test2,Test 3,missing author,missing description,[]
2,http://localhost:3366,http://localhost:3380,None,2024-12-26 18:22:07.113278,2024-12-26 18:22:07.113278,2024-12-26 18:22:07.113278,/test4,Test 4,missing author,missing description,[]


# simulate registry

In [60]:
data = {
    "url": "http://localhost:3381",
    "feed": True,
    "nodes": [
        {
            "url": "http://fantasy.com",
            "flows": []
        }
    ]
}
resp = httpx.post(f"{registry1ip}/connect", json=data)

resp.content

ReadTimeout: timed out

In [ ]:
httpx.get(f"{registry1ip}/map").json()

In [9]:
registry1.terminate()

INFO - 2024-12-26 18:07:41,983 - uvicorn.error - server - Shutting down
INFO - 2024-12-26 18:07:42,085 - uvicorn.error - on - Waiting for application shutdown.
INFO - 2024-12-26 18:07:42,086 - kodo - routes - http://localhost:3366 shutdown now


## HANDSHAKE

The registry is up-and-running.

In [ ]:
resp = httpx.get(registry1ip)
resp.json()

This registry `_cache` is empty. 

In [ ]:
resp = httpx.get(f"{registry1ip}/_cache")
resp.json()

The registry `count` is empty.

In [ ]:
resp = httpx.get(f"{registry1ip}/counts")
resp.json()

# node1 registers with registry1

Start `node1` with a flow `mytest`. The loader must either be a factor string (e.g. `module.path:func`) or a _Callable_.
In this example we use a callable `my_loader` in `myflow`.

```mermaid
stateDiagram
    direction LR
    /mytest --> node1
    node1 --> registry1
```

In [ ]:
import importlib
import flows
importlib.reload(flows)

```json
{
  "url": "string",
  "organization": {},
  "flows": {
    "uid: string": {
      "url": "string",
      "name": "string",
      "description": "string",
      "author": {},
      "tags": []
    },
    "uid: string": {
      "url": "string",
      "name": "string",
      "description": {},
      "author": {},
      "tags": []
    }
  }
}
```

* client: node1
* method: `POST /register`
* server: `http://registry1`
* required parameters
  * url (str)
  * flows (dict)
  * 
```json
{
    "url": str,
    "organization": str,
    "flows": {
        "{url: str}": {
            "url": str,
            "name": str,
            "description": str,
            "tags": str,
            "author": str
        }
    },
    "created": null,
    "modified": null,
    "heartbeat": null,
    "status": "unknown"
}
```

registry1 in response delivers:

```json
{
    "url": "http://localhost:3366",
    "organization": "registry1",
    "node": false,
    "registry": true,
    "provider": false,
    "connection": {},
    "started_at": "2024-12-25T18:03:02.245504",
    "idle": true,
    "now": "2024-12-25T18:03:12.990684",
    "message": [
        "registered node http://localhost:3367 (first visit)"
    ]
}
```

In [ ]:
node1kw = {
    "url": node1ip, 
    "connect": [registry1ip],
    "organization": "node1",
    "node": True, 
    "registry": False,
    "provider": False,
    "reload": False,
    "loader": flows.loader1
}
node1 = Process(target=run_service, kwargs=node1kw)
node1.start()

The node is up-and-running and connected to http://localhost:3366 (registry1)

In [ ]:
resp = httpx.get(node1ip)
resp.json()

There is no such thing as a node `_cache`.

In [ ]:
resp = httpx.get(f"{node1ip}/_cache")
assert resp.status_code == 404

But a flow has been created!

In [ ]:
resp = httpx.get(f"{node1ip}/flows")
resp.json()

Please note that the node itself does not report on any timestamps. This feature is reserved to registries and providers.

The flow is reporting it's `counts`.

In [ ]:
resp = httpx.get(f"{node1ip}/counts")
resp.json()

# Visit the registry

The registry reports timestamps. This is in contrast to the node.

In [ ]:
resp = httpx.get(f"{registry1ip}/flows")
resp.json()

## node1 restart

On restart of the node the registry updates the _modified_ timestamp.

In [ ]:
node1.terminate()

again on `/register` the node sends: 
```json
{
    "url": "http://localhost:3367",
    "organization": "node1",
    "flows": {
        "/mytest": {
            "url": "/mytest",
            "name": "mytest",
            "description": null,
            "tags": null,
            "author": null
        }
    },
    "created": null,
    "modified": null,
    "heartbeat": null,
    "status": "unknown"
}
```

and the registry returns: 
```json
{
    "url": "http://localhost:3366",
    "organization": "registry1",
    "node": false,
    "registry": true,
    "provider": false,
    "connection": {},
    "started_at": "2024-12-25T18:03:02.245504",
    "idle": true,
    "now": "2024-12-25T18:05:13.972890",
    "message": [
        "registered node http://localhost:3367 (seen previously 2024-12-25 18:03:12.989989)"
    ]
}```

In [ ]:
node1 = Process(target=run_service, kwargs=node1kw)
node1.start()

In [ ]:
resp = httpx.get(f"{registry1ip}/flows")
resp.json()

On registry restart the flow collection is retrieved from cache. Both the _created_ and _modified_ timestamp remain unchanged.

In [ ]:
registry1.terminate()

In [ ]:
# enable cache!
registry1kw["reset"] = False
registry1 = Process(target=run_service, kwargs=registry1kw)
registry1.start()

In [ ]:
resp = httpx.get(f"{registry1ip}/flows")
resp.json()

# REGISTRY2 STARTUP

Create `registry2` and connect with `registry1`.

```mermaid
stateDiagram
    direction LR
    /mytest --> node1
    node1 --> registry1
    registry2 --> registry1
```

`registry2` at startup connects to `registry1` and POST /connect: 
```json
{
    "url": "http://localhost:3368",
    "organization": "registry2",
    "feed": true,
    "nodes": {}
}
```

`registry1` returns
```json
{
    "providers": {
        "url": "http://localhost:3366",
        "organization": "registry1",
        "feed": true,
        "nodes": {
            "http://localhost:3367": {
                "url": "http://localhost:3367",
                "organization": "node1",
                "flows": {
                    "/mytest": {
                        "url": "/mytest",
                        "name": "mytest",
                        "description": null,
                        "tags": null,
                        "author": null
                    }
                },
                "created": "2024-12-25T18:03:12.989989",
                "modified": "2024-12-25T18:05:13.972356",
                "heartbeat": "2024-12-25T18:05:13.972356",
                "status": "unknown"
            }
        }
    },
    "message": [
        "registry http://localhost:3368 connected (first visit)"
    ]
}
```

In [ ]:
registry2kw = {
    "url": registry2ip, 
    "connect": [registry1ip],
    "organization": "registry2",
    "node": False, 
    "registry": True,
    # initialise the cache, which we will use later
    "cache": "./registry2.json",
    # start from scratch
    "reset": True
}
registry2 = Process(target=run_service, kwargs=registry2kw)
registry2.start()

# HANDSHAKE

The registry is up-and-running and connected to `registry1`.

In [ ]:
resp = httpx.get(registry2ip)
resp.json()

Both registries `registry1` and `registry2` are in-sync.

In [ ]:
resp1 = httpx.get(f"{registry1ip}/flows")
resp2 = httpx.get(f"{registry2ip}/flows")

In [ ]:
item1 = resp1.json()["items"][0]
item2 = resp2.json()["items"][0]

# remove the registry masquerade
item1.pop("registry")
item2.pop("registry")

assert item1 == item2

# Node2 connects to registry2
## Node2 joins the mesh

```mermaid
stateDiagram
    direction LR
    /mytest --> node1
    node1 --> registry1
    registry2 --> registry1
    node2 --> registry2
    /mytest2 --> node2
```

node2 `POST http://registry2/register` with

```json
{
    "url": "http://localhost:3369",
    "organization": "node2",
    "flows": {
        "/mytest2": {
            "url": "/mytest2",
            "name": "mytest2",
            "description": null,
            "tags": null,
            "author": null
        }
    },
    "created": null,
    "modified": null,
    "heartbeat": null,
    "status": "unknown"
}
```

registry2 responds with

```json
{
    "url": "http://localhost:3368",
    "organization": "registry2",
    "node": false,
    "registry": true,
    "provider": false,
    "connection": {
        "http://localhost:3366": "2024-12-25T18:07:39.033999"
    },
    "started_at": "2024-12-25T18:07:39.013209",
    "idle": false,
    "now": "2024-12-25T18:35:36.659965",
    "message": [
        "registered node http://localhost:3369 (first visit)",
        "feed forward node to http://localhost:3366"
    ]
}
```

registry2 feed forwards the node update to registry1 with `POST http://registry1/update/node`

```json
{
    "url": "http://localhost:3368",
    "organization": "registry2",
    "feed": false,
    "nodes": {
        "http://localhost:3369": {
            "url": "http://localhost:3369",
            "organization": "node2",
            "flows": {
                "/mytest2": {
                    "url": "/mytest2",
                    "name": "mytest2",
                    "description": null,
                    "tags": null,
                    "author": null
                }
            },
            "created": "2024-12-25T18:35:36.651614",
            "modified": "2024-12-25T18:35:36.651614",
            "heartbeat": "2024-12-25T18:35:36.651614",
            "status": "unknown"
        }
    }
}
```

response of `registry1` is

```json
{
    "url": "http://localhost:3366",
    "organization": "registry1",
    "node": false,
    "registry": true,
    "provider": false,
    "connection": {},
    "started_at": "2024-12-25T18:07:18.806234",
    "idle": true,
    "now": "2024-12-25T18:35:36.693275",
    "message": [
        "created node http://localhost:3368"
    ]
}
```

In [ ]:
importlib.reload(flows)

In [ ]:
node2kw = {
    "url": node2ip, 
    "connect": [registry2ip],
    "organization": "node2",
    "node": True, 
    "registry": False,
    "provider": False,
    "reload": False,
    "loader": flows.loader2
}
node2 = Process(target=run_service, kwargs=node2kw)
node2.start()

The node is up-and-running and connected to http://localhost:3368 (registry2)

In [ ]:
resp = httpx.get(node2ip)
resp.json()

The flow has been created!

In [ ]:
resp = httpx.get(f"{node2ip}/flows")
resp.json()

Again the node does not report on any timestamps.

# Visit all registries

In [ ]:
resp1 = httpx.get(f"{registry1ip}/flows")
resp2 = httpx.get(f"{registry2ip}/flows")

In [ ]:
resp1.json()

In [ ]:
resp2.json()

In [ ]:
from IPython.display import display, HTML

In [ ]:
resp1 = httpx.get(f"{registry1ip}/flows?format=html")
display(HTML(resp1.content.decode("utf-8")))

In [ ]:
resp1 = httpx.get(f"{registry2ip}/flows?format=html")
display(HTML(resp1.content.decode("utf-8")))

# Node updates

When a node reconnects, the news is broadcasted from the sourcing registry to all peers.

In [ ]:
node1.terminate()

In [ ]:
node1kw = {
    "url": node1ip, 
    "connect": [registry1ip],
    "organization": "node1",
    "node": True, 
    "registry": False,
    "provider": False,
    "reload": False,
    "loader": flows.loader3
}

node1 connects to registry1
```json
{
    "url": "http://localhost:3367",
    "organization": "node1",
    "flows": {
        "/mytest2": {
            "url": "/mytest2",
            "name": "mytest2",
            "description": "This is a testing flow",
            "tags": null,
            "author": "michi.rau@gmail.com"
        }
    },
    "created": null,
    "modified": null,
    "heartbeat": null,
    "status": "unknown"
}
```

registry1 responds with
```json
{
    "url": "http://localhost:3366",
    "organization": "registry1",
    "node": false,
    "registry": true,
    "provider": false,
    "connection": {},
    "started_at": "2024-12-25T18:07:18.806234",
    "idle": false,
    "now": "2024-12-25T18:44:02.640826",
    "message": [
        "registered node http://localhost:3367 (seen previously 2024-12-25 18:03:12.989989)",
        "feed forward node to http://localhost:3368"
    ]
}
```

registry1 broadcasts the update of node1 to registry2
```json
{
    "url": "http://localhost:3366",
    "organization": "registry1",
    "feed": false,
    "nodes": {
        "http://localhost:3367": {
            "url": "http://localhost:3367",
            "organization": "node1",
            "flows": {
                "/mytest2": {
                    "url": "/mytest2",
                    "name": "mytest2",
                    "description": "This is a testing flow",
                    "tags": null,
                    "author": "michi.rau@gmail.com"
                }
            },
            "created": "2024-12-25T18:03:12.989989",
            "modified": "2024-12-25T18:44:02.640011",
            "heartbeat": "2024-12-25T18:44:02.640011",
            "status": "unknown"
        }
    }
}
```

registry2 responds with
```json
{
    "url": "http://localhost:3368",
    "organization": "registry2",
    "node": false,
    "registry": true,
    "provider": false,
    "connection": {
        "http://localhost:3366": "2024-12-25T18:35:36.694219"
    },
    "started_at": "2024-12-25T18:07:39.013209",
    "idle": true,
    "now": "2024-12-25T18:44:02.660559",
    "message": [
        "updated node http://localhost:3366"
    ]
}
```

In [ ]:
node1 = Process(target=run_service, kwargs=node1kw)
node1.start()

## revisit registries

In [ ]:
resp1 = httpx.get(f"{registry1ip}/flows?format=html")
display(HTML(resp1.content.decode("utf-8")))

In [ ]:
resp2 = httpx.get(f"{registry2ip}/flows?format=html")
display(HTML(resp2.content.decode("utf-8")))

# TERMINATION

In [ ]:
registry1.terminate()

In [ ]:
node1.terminate()

In [ ]:
registry2.terminate()

In [ ]:
node2.terminate()